In [76]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import random

# 隨機選擇 User-Agent 防止被網站屏蔽
headers = {
    'User-Agent': random.choice([
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'
    ])
}

# Step 1: 從 Wikipedia 獲取電影名稱與 Rotten Tomatoes 連結
def get_movie_urls_from_wikipedia(url):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"無法連接到 Wikipedia 頁面, 狀態碼: {response.status_code}")
        return [], []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_urls = []
    movie_titles = []
    
    for link in soup.find_all('a', href=True):
        if "rottentomatoes.com/m/" in link['href']:
            movie_urls.append(link['href'])
            movie_titles.append(link.text.strip())
    
    return movie_titles, movie_urls

# Step 2: 將結果存入 CSV 文件
def save_to_csv(movie_titles, movie_urls, output_file):
    # 創建 DataFrame
    df = pd.DataFrame({
        'Movie Title': movie_titles,
        'Rotten Tomatoes URL': movie_urls
    })
    
    # 保存到 CSV 文件
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"資料已保存到 {output_file}")

# 執行流程
wikipedia_url = "https://en.wikipedia.org/wiki/List_of_films_with_a_0%25_rating_on_Rotten_Tomatoes"
output_directory = r"C:\Users\User\Desktop\學\大學\python"  # 更改為你的輸出路徑
output_file = os.path.join(output_directory, "movie_titles_and_urls.csv")

# 獲取電影名稱和連結
movie_titles, movie_urls = get_movie_urls_from_wikipedia(wikipedia_url)

# 保存結果到 CSV
if movie_titles and movie_urls:
    save_to_csv(movie_titles, movie_urls, output_file)
else:
    print("未能提取到任何電影資料")


資料已保存到 C:\Users\User\Desktop\學\大學\python\movie_titles_and_urls.csv


In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
import random
from datetime import datetime

# 隨機選擇 User-Agent 防止被網站屏蔽
headers = {
    'User-Agent': random.choice([
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'
    ])
}

# Step 1: 爬取導演名稱與導演專業連結
def get_director_info_from_rottentomatoes(movie_url):
    response = requests.get(movie_url, headers=headers)
    
    if response.status_code != 200:
        print(f"無法連接到電影頁面: {movie_url}, 狀態碼: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # 查找 JSON-LD 數據
    script_tag = soup.find('script', type='application/ld+json')
    if not script_tag:
        print(f"未找到 JSON-LD 數據: {movie_url}")
        return None

    # 解析 JSON 數據
    movie_data = json.loads(script_tag.string)

    # 提取導演名稱和連結
    directors = movie_data.get('director', [])
    directors_info = []

    for director in directors:
        director_name = director.get('name', 'Unknown Director')
        director_url = director.get('sameAs', 'Unknown URL')
        
        # 爬取導演個人資料
        director_bio_info = get_director_bio_info(director_url)
        if director_bio_info:
            director_birthday, director_birthplace, highest_rated, lowest_rated, age = director_bio_info
        else:
            director_birthday, director_birthplace, highest_rated, lowest_rated, age = 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'
        
        directors_info.append({
            'Movie Title': movie_data.get('name', 'Unknown Movie'),
            'Director Name': director_name,
            'Director URL': director_url,
            'Birthday': director_birthday,
            'Birthplace': director_birthplace,
            'Highest Rated': highest_rated,
            'Lowest Rated': lowest_rated,
            'Age': age
        })
    
    return directors_info

# Step 2: 爬取導演的生日、出生地、最高與最低分作品
def get_director_bio_info(director_url):
    response = requests.get(director_url, headers=headers)
    
    if response.status_code != 200:
        print(f"無法連接到導演頁面: {director_url}, 狀態碼: {response.status_code}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # 提取生日
    birthday_tag = soup.find('p', {'data-qa': 'celebrity-bio-bday'})
    birthday = birthday_tag.text.strip().replace("Birthday:", "").strip() if birthday_tag else 'N/A'
    # 計算年齡
    age = calculate_age(birthday) if birthday != 'N/A' else 'N/A'
    
    # 提取出生地
    birthplace_tag = soup.find('p', {'data-qa': 'celebrity-bio-birthplace'})
    birthplace = birthplace_tag.text.strip().replace("Birthplace:", "").strip() if birthplace_tag else 'N/A'

    # 提取最高與最低分作品
    highest_rated_tag = soup.find('p', {'data-qa': 'celebrity-bio-highest-rated'})
    highest_rated = highest_rated_tag.find('a').text.strip() if highest_rated_tag and highest_rated_tag.find('a') else 'N/A'

    lowest_rated_tag = soup.find('p', {'data-qa': 'celebrity-bio-lowest-rated'})
    lowest_rated = lowest_rated_tag.find('a').text.strip() if lowest_rated_tag and lowest_rated_tag.find('a') else 'N/A'
    
    return birthday, birthplace, highest_rated, lowest_rated, age

# 計算年齡
def calculate_age(birthday):
    try:
        birth_date = datetime.strptime(birthday, "%b %d, %Y")
        today = datetime.now()
        return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    except ValueError:
        return 'N/A'

# Step 3: 從 CSV 中讀取電影 URL，並獲取導演資訊
def extract_director_info_from_csv(input_file, output_file):
    # 從 CSV 讀取電影標題和連結
    df = pd.read_csv(input_file)

    directors_info = []

    for index, row in df.iterrows():
        movie_title = row['Movie Title']
        movie_url = row['Rotten Tomatoes URL']
        print(f"正在處理電影: {movie_title}")

        directors = get_director_info_from_rottentomatoes(movie_url)

        if directors:
            for director_data in directors:
                directors_info.append(director_data)

    # 將導演資訊保存到新的 CSV 文件
    directors_df = pd.DataFrame(directors_info)
    directors_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"導演資料已保存到 {output_file}")

# 執行流程
input_csv_file = r"C:\Users\User\Desktop\學\大學\python\movie_titles_and_urls.csv"  # 更改為你的輸入路徑
output_csv_file = r"C:\Users\User\Desktop\學\大學\python\directors_info.csv"  # 更改為你的輸出路徑

# 提取導演資訊並保存
extract_director_info_from_csv(input_csv_file, output_csv_file)


正在處理電影: "Staying Alive"
正在處理電影: "Bolero (1984)"
正在處理電影: "Police Academy 4: Citizens on Patrol"
正在處理電影: "Problem Child (1990)"
正在處理電影: "Highlander 2: The Quickening (1991)"
正在處理電影: "Return to the Blue Lagoon (1991)"
正在處理電影: "Folks! (1992)"
正在處理電影: "Look Who's Talking Now (1993)"
正在處理電影: "Wagons East! (1994)"
正在處理電影: "Simon Sez (1999)"
正在處理電影: "3 Strikes (2000)"
正在處理電影: "Ballistic: Ecks vs. Sever (2002)"
正在處理電影: "Killing Me Softly"
正在處理電影: "Merci Docteur Rey (2002)"
正在處理電影: "Pinocchio (2002)"
正在處理電影: "Derailed"
正在處理電影: "National Lampoon's Gold Diggers (2004)"
正在處理電影: "Superbabies: Baby Geniuses 2 (2004)"
正在處理電影: "Constellation"
正在處理電影: "Redline (2007)"
正在處理電影: "Scar (2007)"
正在處理電影: "One Missed Call"
正在處理電影: "Homecoming"
正在處理電影: "Stolen"
正在處理電影: "Transylmania (2009)"
正在處理電影: "The Nutcracker in 3D (2010)"
正在處理電影: "Beneath the Darkness"
正在處理電影: "Dark Tide (2012)"
正在處理電影: "A Thousand Words"
正在處理電影: "Left Behind"
正在處理電影: "The Ridiculous 6 (2015)"
正在處理電影: "Cabin Fever"
正在處理電影: "Dark Crimes (20

In [90]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

# 隨機選擇 User-Agent 防止被網站屏蔽
headers = {
    'User-Agent': random.choice([
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'
    ])
}

# 爬取導演的電影或TV作品列表
def get_director_filmography(director_url):
    response = requests.get(director_url, headers=headers)
    
    if response.status_code != 200:
        print(f"無法連接到導演頁面: {director_url}, 狀態碼: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    filmography = []

    for category in ['movies', 'tv']:
        works = soup.select(f'table[data-qa="celebrity-filmography-{category}"] tbody > tr')

        for work in works:
            title = work.get('data-title', '未找到標題')
            
            # 判斷是否有評分
            tomatometer_raw = work.get('data-tomatometer', '')
            if tomatometer_raw == "0":
                tomatometer = "No Score Yet"  # 無評分時
            else:
                tomatometer = f"{tomatometer_raw}%"  # 有具體評分
            
            audience_score_raw = work.get('data-audiencescore', '')
            if audience_score_raw == "0":
                audience_score = "No Score Yet"  # 無評分時
            else:
                audience_score = f"{audience_score_raw}%"  # 有具體評分
            
            # 處理票房和年份
            box_office = work.get('data-boxoffice', '未找到票房')
            year_raw = work.get('data-appearance-year', work.get('data-year', '未找到年份'))
            year = process_year(year_raw)
            
            # 抓取導演在該作品的職位
            credits = work.find('td', class_='celebrity-filmography__credits').text.strip() if work.find('td', class_='celebrity-filmography__credits') else '未找到職位'

            # 將作品加入列表，並標註是電影還是電視劇
            filmography.append({
                'title': title,
                'year': year,
                'tomatometer': tomatometer,  # 保留評分或無評分的狀態
                'audience_score': audience_score,
                'box_office': box_office,
                'credits': credits,
                'category': 'Movie' if category == 'movies' else 'TV Show'  # 新增分類
            })
    
    return filmography
    
def process_year(year_raw):
    """
    處理年份數據，包括單一年份、範圍年份和多個年份。
    """
    # 檢查是否包含多個年份
    if isinstance(year_raw, list):
        # 多個年份時，將年份列表轉為用逗號分隔的字符串
        return ", ".join(year_raw)
    
    # 檢查是否是年份範圍，如 "[2022-2024]"
    if year_raw.startswith("[") and year_raw.endswith("]"):
        # 去除方括號，然後分割年份範圍
        year_range = year_raw.strip("[]").split('-')
        start_year = year_range[0]
        end_year = year_range[1] if len(year_range) > 1 else start_year  # 如果只有一個年份，則將其視為起始年份
        return f"{start_year}-{end_year}"
    
    # 單一年份直接返回
    return year_raw



# 讀取導演 URL 並爬取他們的作品列表
def extract_movies_from_directors(input_file, output_file):
    # 讀取 CSV 文件中導演的 URL
    df = pd.read_csv(input_file)
    all_movies = []

    for index, row in df.iterrows():
        director_name = row['Director Name']
        director_url = row['Director URL']
        print(f"正在處理導演: {director_name}")

        # 爬取該導演的電影或TV列表
        movies = get_director_filmography(director_url)
        
        # 將導演的每部作品與其職位一起加入列表
        for movie in movies:
            movie['Director Name'] = director_name  # 添加導演名稱
            all_movies.append(movie)

    # 將作品列表保存為 CSV 文件
    movies_df = pd.DataFrame(all_movies)
    movies_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"電影或劇集資料已保存到 {output_file}")

# 執行流程
input_csv_file = r"C:\Users\User\Desktop\學\大學\python\directors_info.csv"  # 導演資料的輸入文件
output_csv_file = r"C:\Users\User\Desktop\學\大學\python\movies_from_directors.csv"  # 將電影資料保存到此文件

# 提取電影資訊並保存
extract_movies_from_directors(input_csv_file, output_csv_file)


正在處理導演: Sylvester Stallone
正在處理導演: John Derek
正在處理導演: Jim Drake
正在處理導演: Dennis Dugan
正在處理導演: Russell Mulcahy
正在處理導演: William Graham
正在處理導演: Ted Kotcheff
正在處理導演: Tom Ropelewski
正在處理導演: Peter Markle
正在處理導演: Kevin Alyn Elders
正在處理導演: DJ Pooh
正在處理導演: Wych Kaosayananda
正在處理導演: Chen Kaige
正在處理導演: Andrew Litvack
正在處理導演: Roberto Benigni
正在處理導演: Bob Misiorowski
正在處理導演: Gary Preisler
正在處理導演: Bob Clark
正在處理導演: Jordan Walker-Pearlman
正在處理導演: Andy Cheng
正在處理導演: Jed Weintrob
正在處理導演: Éric Valette
正在處理導演: Morgan J. Freeman
正在處理導演: Anders Anderson
正在處理導演: David Hillenbrand
正在處理導演: Scott Hillenbrand
正在處理導演: Andrey Konchalovskiy
正在處理導演: Martin Guigui
正在處理導演: John Stockwell
正在處理導演: Brian Robbins
正在處理導演: Vic Armstrong
正在處理導演: Frank Coraci
正在處理導演: Travis Zariwny
正在處理導演: Alexandros Avranas
正在處理導演: D.J. Caruso
正在處理導演: Stewart Hendler
正在處理導演: Max Adams
正在處理導演: Simon West
正在處理導演: Kevin Connolly
正在處理導演: Mathew Cullen
正在處理導演: Vincent Kesteloot
正在處理導演: Ben Stassen
正在處理導演: Will Forbes
正在處理導演: Olivier Megaton
正在處理導演